In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
device = 'cuda' if torch.cuda.is_available else 'cpu'
print(device)

cuda


In [14]:
with open('AlisInWonderland.txt', 'r', encoding = 'UTF-8') as file:
    raw_data = file.read()
words = raw_data.split()
chars = sorted(set(raw_data))
chtoin = {c:i for i,c in enumerate(chars)} ## encoding one letter 
intoch = {i:c for i,c in enumerate(chars)}
encode_seq = lambda i : [chtoin[ch] for ch in i] ### take sequ and return indexes
decode_seq = lambda ch :[''.join(intoch[ind]) for ind in ch] ## take indexs and return "string"

In [177]:
batchs = 4
block = 8
enc_data = torch.tensor(encode_seq(raw_data), dtype = torch.long)
train_data = enc_data[:int(0.8 * len(enc_data))]
val_data = enc_data[int(0.8 * len(enc_data)):]

In [16]:
inp = train_data[:block]
targ = train_data[1:block+1]
for i in range(block):
    context = inp[:i+1]
    target = targ[i]
    print(f"when the input sequ is {context}||| Target is :{target}")

when the input sequ is tensor([83])||| Target is :41
when the input sequ is tensor([83, 41])||| Target is :58
when the input sequ is tensor([83, 41, 58])||| Target is :55
when the input sequ is tensor([83, 41, 58, 55])||| Target is :1
when the input sequ is tensor([83, 41, 58, 55,  1])||| Target is :37


In [17]:
def get_batch(data_type):
    if data_type == 'train':
        data = train_data
    else:
        data = val_data

    temp = torch.randint(len(data) - block, (batchs,))### will return list of 4 numbers like [5112,156,198,1555]
    x_b = torch.stack([data[i:i+block] for i in temp])
    y_b = torch.stack([data[i+1:i+block+1] for i in temp])
    return x_b, y_b
x_b,y_b = get_batch('train')
print(x_b)

for i in range(batchs):
    for j in range(block):
        print(f'when the input is {x_b[i][:j+1]} the target shoudl be: {y_b[i][j]}')

tensor([[ 1, 69, 65,  0, 72],
        [70, 58, 59, 69,  1],
        [58, 65, 71, 57, 58],
        [ 1, 30,  1, 53, 51]])
when the input is tensor([1]) the target shoudl be: 69
when the input is tensor([ 1, 69]) the target shoudl be: 65
when the input is tensor([ 1, 69, 65]) the target shoudl be: 0
when the input is tensor([ 1, 69, 65,  0]) the target shoudl be: 72
when the input is tensor([ 1, 69, 65,  0, 72]) the target shoudl be: 59
when the input is tensor([70]) the target shoudl be: 58
when the input is tensor([70, 58]) the target shoudl be: 59
when the input is tensor([70, 58, 59]) the target shoudl be: 69
when the input is tensor([70, 58, 59, 69]) the target shoudl be: 1
when the input is tensor([70, 58, 59, 69,  1]) the target shoudl be: 51
when the input is tensor([58]) the target shoudl be: 65
when the input is tensor([58, 65]) the target shoudl be: 71
when the input is tensor([58, 65, 71]) the target shoudl be: 57
when the input is tensor([58, 65, 71, 57]) the target shoudl b

### after we encdoing all the text depending on the character level, right now we will get up little pit, so we will create bigram model that is create embedder at the first for actoring all the letters in each token with len(data) wieghts which called logits and this logits will be for every letter in every batching in every block

In [18]:
class bigram(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, xs, ys = None):
        logits = self.token_embedding_table(xs)
        if ys == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            ys = ys.view(B*T)
            loss = F.cross_entropy(logits, ys)
        return logits, loss
    def gen(self, xs, max_t):
            for i in range(max_t):
                logits, loss = self.forward(xs)
                logits = logits[:, -1, :] ## i want to take the last element from every batchs
                probs = F.softmax(logits, dim = 1)
                next_x = torch.multinomial(probs, num_samples = 1)
                xs = torch.cat((xs,next_x), dim = 1)
            return xs
                        


### in this simple model i get the prediction depending on just the last letter i mean i get V depending on \n (the history is not used)
after that i will upgrading the model and make it take all the sequence and get the result depending on it


In [19]:
bm = bigram(len(chars))
start_x = torch.zeros((1,1), dtype = torch.long)
rr = ''.join(decode_seq(bm.gen(start_x, 500)[0].tolist()))
print(rr)


f.X75Afx'PGUhnLc(Q0Q!T_?LF;aK3uINPw”kvOsLLJ’k4H*c7B—Ygmatt8“(-kJ;ùr‘qOL P*”(07q0Qb,8eIf#m,QS4z—’lS1—LhbH8Ea—N-vY__BL!VxxUzA51““.4KWXm;a’Z’aIr‘HJOWlBLMoV1'Hl;GtU?:-5:C?1— t.—,u‘Nbz‘Sh‘0m—b?bq7 1o”.Qe##”ùxW]Xv#h0uULB7 51LnlGW-‘:M3)OojGh0m)X[KQGDK2hC—t.sda’KKxIi!u32K3]?5A﻿”sV0d),8U0!lsDgLH_hxXaz—Ce“3!SZUDJ*:_3’TGI“aEaf— d15l,D_ToDGRZI#DggW?hmH.qpnM“RItU﻿TP﻿XNtJ“yYg1;0﻿S8NzM7 F(KCj[Nlgm‘7LSK3y_U!rfSz1!r‘I(ydNDHf’kClF(v‘tUpGYGum—ZK﻿QhLP2Jvv_OaU ClZmPùZNPAGe—Usg.—FKjxUQuv(k SKLvfR Ct]’kyF;AVb??oT483!p


In [20]:
batchs = 32
optimizer = torch.optim.AdamW(bm.parameters(), lr = 0.0001)
for steps in range(100):
    xb, yb = get_batch('train')
    logtis, loss = bm(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

4.950274467468262


# Simplest possible model

In [21]:
start_x = torch.zeros((1,1), dtype = torch.long)
rr = ''.join(decode_seq(bm.gen(start_x, 500)[0].tolist()))
print(rr)


DjvkJ*D:l*VbGu‘;﻿5BmUz#—mKnh‘Vdi.,[3[Kjrf5SP0﻿y:’khLKF[3qD_LrOL!8G5Ay8o
OLmUSTYg1Cj21tuOa7c,
—mcxU.WBRMXOr1;tZJA.,
Z’kV NH”jL[gjNa*qP1hcùb0sDlIj;’kV0gIe﻿JITzgP-o-v#YuRC-tyo-mw.vftfl'RgQT,-eoWX!:)t.gX:hAV:RKjx—OucJnv#:s'vSKqb);07H5I#XHyqBRs“”rfF;#:k4VQuWBT Ehm)BoN[sLS_) [u)‘IvFBB:Cd)﻿-rqIwSGLeZ“GG)﻿]?ClmA
vH*‘.d-vJ;v.x1!#”sLzqW!
_ZnOL-v#m PPJ7_ZURGka1L‘Lh]8OxJBd-Ua3cXù(xF(a0M])Rv#“r)XroKWBCYd'8w,cf.g’hùQ’jLh5!M,-.j—Yu﻿ZN4cJl
fda,'dAEO5tB]mSh2EAuRgNrlZnoW!abHkiT“(vrs‘IyDp_J‘LMCL“5X_r‘Hù.—F0;pnimHm


In [66]:
T = block
a = torch.tril(torch.ones(T,T))
a /= torch.sum(a,1, keepdim = True)
print(a)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


In [73]:
T = block
a = torch.tril(torch.ones(T,T))
a = torch.zeros((T,T)).masked_fill(a == 0,float('-inf'))
a = F.softmax(a,dim = -1)
a

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [100]:
a = torch.tril(torch.ones(T, T))

out = {}
for i in range(len(a)):
    out [i] = a[i] / (i+1)
out

{0: tensor([1., 0., 0., 0., 0., 0., 0., 0.]),
 1: tensor([0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]),
 2: tensor([0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]),
 3: tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000]),
 4: tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000]),
 5: tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000]),
 6: tensor([0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000]),
 7: tensor([0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250])}

The Next cell will be in python file 

In [212]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 2000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------




with open('AlisInWonderland.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.21218 M parameters
step 0: train loss 4.6659, val loss 4.6722
step 100: train loss 2.6334, val loss 2.6173
step 200: train loss 2.4894, val loss 2.4732
step 300: train loss 2.3907, val loss 2.3739
step 400: train loss 2.2974, val loss 2.2912
step 500: train loss 2.2142, val loss 2.2119
step 600: train loss 2.1440, val loss 2.1438
step 700: train loss 2.0957, val loss 2.0991
step 800: train loss 2.0230, val loss 2.0281
step 900: train loss 1.9658, val loss 1.9854
step 1000: train loss 1.9133, val loss 1.9316
step 1100: train loss 1.8854, val loss 1.9126
step 1200: train loss 1.8328, val loss 1.8757
step 1300: train loss 1.8077, val loss 1.8532
step 1400: train loss 1.7855, val loss 1.8296
step 1500: train loss 1.7546, val loss 1.7941
step 1600: train loss 1.7318, val loss 1.7961
step 1700: train loss 1.7087, val loss 1.7697
step 1800: train loss 1.6821, val loss 1.7552
step 1900: train loss 1.6552, val loss 1.7302
step 1999: train loss 1.6465, val loss 1.7099

with
inte hums as all fl